Import required libraries

In [2]:
# Required to convert .py file into .ipynb
!pip install jupyter-dash  
# To run Dash on Jupyter
from jupyter_dash import JupyterDash  

import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

Read the airline data into pandas dataframe

In [3]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

Create a dash application

In [4]:
app = dash.Dash(__name__)

Create an app layout

In [5]:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                    options = [{'label':'All Sites', 'value':'ALL'},
                                    {'label': 'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                    {'label': 'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                    {'label': 'KSC LC-39A', 'value':'KSC LC-39A'},
                                    {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40'}],
                                    value = 'ALL', placeholder = "Select launch site",
                                    searchable = True),
                                html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0', 100: '100'},
                                    value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

TASK 2:<br>
Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

In [6]:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, 
        values='class', 
        names='Launch Site', 
        title='Overall Successful vs. Failed launches')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df = filtered_df.groupby(['class'])['class'].count().reset_index(name='success')
        fig = px.pie(filtered_df, 
        values='success', 
        names='class', 
        title='Site-specific Successful vs. Failed launches')
        return fig

TASK 4:<br>
Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

In [7]:
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value'))
def get_scatter_chart(entered_site, payload):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, 
        x='Payload Mass (kg)', 
        y='class', 
        color="Booster Version Category",
        title='Successful vs. Failed launches by Payload & Booster Version')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        fig = px.scatter(filtered_df, 
        x='Payload Mass (kg)', 
        y='class', 
        color="Booster Version Category",
        title='Site-specific Successful vs. Failed launches by Payload & Booster Version')
        return fig

Run the app

In [ ]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Mar/2023 12:58:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2023 12:58:53] "GET /_dash-component-suites/dash/deps/react@16.v2_8_1m1677662184.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2023 12:58:53] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_8_1m1677662184.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2023 12:58:53] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_8_1m1677662184.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2023 12:58:53] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_8_1m1677662184.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2023 12:58:53] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_8_1m1677662184.min.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2023 12:58:53] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_8_0m1677662184.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Mar/2023 12:58:53] "GET /_dash-component-suites/d